Computational Theory Problems

In [1]:
import numpy as np

Problem 1: Binary Words and Operations

In [2]:
def Parity(x, y, z):
    return (x ^ y ^ z)

In [3]:
def Ch(x, y, z):
    return ((x & y) ^ (x & z))

In [4]:
def Maj(x, y, z):
    return ((x & y) ^ (x & z) ^ (y & z))

In [ ]:
def ROTR(x, n, w):
    return ((x >> n) | (x << (w - n )))

In [6]:
def Sigma0(x):
    return ROTR(x, 2) ^ ROTR(x, 13) ^ ROTR(x, 22)

In [7]:
def Sigma1(x):
    return ROTR(x, 6) ^ ROTR(x, 11) ^ ROTR(x, 25)

In [8]:
def SHR(x, n):
    return x >> n

In [9]:
def sigma0(x):
    return ROTR(x, 7) ^ ROTR(x, 18) ^ SHR(x, 3)

In [10]:
def sigma0(x):
    return ROTR(x, 17) ^ ROTR(x, 19) ^ SHR(x, 10)

Problem 2: Fractional Parts of Cube Roots

In [ ]:
def primes(n):
    

Problem 3: Padding

Problem 4: Hashes

Problem 5: Passwords